In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_Train=pd.read_csv("Train.csv",usecols=['AuthorID','Text'])
df_Test=pd.read_csv("Test.csv",usecols=['AuthorID','Text'])
df_Val=pd.read_csv("Valid.csv",usecols=['AuthorID','Text'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_Train.head()

,AuthorID,Text
0,965226,<NAME/> McHargue will be backing up Jerry Wats...
1,543719,I'm sorry I missed your call. I know you are b...
2,x16198468117121,I'm doing fine. We can't control all events e...
3,997109,"<NAME/>, thank you very much for the invite, b..."
4,464635,I was just looking at the confirm for April an...


In [ ]:
df_Train['Text']=df_Train['Text'].str.lower()
df_Test['Text']=df_Test['Text'].str.lower()
df_Val['Text']=df_Val['Text'].str.lower()

In [ ]:
df_Train.head()
df_Train['Text'][0]

'<name/> mchargue will be backing up jerry watson this weekend.   she will be another contact person available to us if we cannot contact jerry.  her numbers:cell:   813-293-0700home:   727-522-6741pager:   813-266-8044remember to call her this weekend with any frontera changes.thanks,<name/>'

In [ ]:
import re

def remove_html_tags(text):
    pattern = re.compile(r'<.*?>')
    return pattern.sub(r'', text)

In [ ]:
df_Train['Text']=df_Train['Text'].apply(remove_html_tags)
df_Test['Text']=df_Test['Text'].apply(remove_html_tags)
df_Val['Text']=df_Val['Text'].apply(remove_html_tags)

In [ ]:
df_Train.head()
df_Train['Text'][0]

' mchargue will be backing up jerry watson this weekend.   she will be another contact person available to us if we cannot contact jerry.  her numbers:cell:   813-293-0700home:   727-522-6741pager:   813-266-8044remember to call her this weekend with any frontera changes.thanks,'

In [ ]:
import re
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [ ]:
df_Train['Text']=df_Train['Text'].apply(remove_url)
df_Test['Text']=df_Test['Text'].apply(remove_url)
df_Val['Text']=df_Val['Text'].apply(remove_url)

In [ ]:
df_Train.head()
df_Train['Text'][0]

' mchargue will be backing up jerry watson this weekend.   she will be another contact person available to us if we cannot contact jerry.  her numbers:cell:   813-293-0700home:   727-522-6741pager:   813-266-8044remember to call her this weekend with any frontera changes.thanks,'

## Label Encoding the Author ID's

In [ ]:
unique_authors=list(df_Train['AuthorID'].unique())
hashmap={v:i for i,v in enumerate(unique_authors)}

In [ ]:
df_Train['AuthorIDMod']=df_Train['AuthorID'].apply(lambda x: hashmap[x] if x in hashmap else 26)
df_Val['AuthorIDMod']=df_Val['AuthorID'].apply(lambda x: hashmap[x] if x in hashmap else 26)
df_Test['AuthorIDMod']=df_Test['AuthorID'].apply(lambda x: hashmap[x] if x in hashmap else 26)

In [ ]:
train=df_Train[['Text','AuthorIDMod']].rename(columns={'Text':'text','AuthorIDMod':'labels'})
test=df_Test[['Text','AuthorIDMod']].rename(columns={'Text':'text','AuthorIDMod':'labels'})
val=df_Val[['Text','AuthorIDMod']].rename(columns={'Text':'text','AuthorIDMod':'labels'})

In [ ]:
train.to_csv('train_.csv',index=False)
test.to_csv('test_.csv',index=False)
val.to_csv('val_.csv',index=False)

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv",data_files={"train":"train_.csv","val":"val_.csv","test":"test_.csv"})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import BertTokenizer

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def tokenize_function(examples):
    text = examples["text"]

    # Check if text is not null and not empty
    if text is not None and text.strip() != "":
        # Use tokenizer on non-empty text
        tokens = tokenizer(text, padding="max_length", truncation=True)

        # Check if 'token_type_ids' is present in the tokenizer output
        if 'token_type_ids' not in tokens:
            # Add 'token_type_ids' if missing (e.g., for models like BERT)
            tokens['token_type_ids'] = [0] * len(tokens['input_ids'])

        return tokens
    else:
        # Return some default value or handle the case as needed
        return {"input_ids": [], "attention_mask": [], "token_type_ids": []}

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenized_datasets = dataset.map(tokenize_function)



Map:   0%|          | 0/3001 [00:00<?, ? examples/s]

Map:   0%|          | 0/601 [00:00<?, ? examples/s]

Map:   0%|          | 0/634 [00:00<?, ? examples/s]

In [ ]:
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["val"]

In [ ]:
from transformers import  BertForSequenceClassification
num_labels = 27

model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=27)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate

metric = evaluate.load(
    "accuracy",
    "precision",


)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
training_args = TrainingArguments(output_dir="/content/Docs", evaluation_strategy="epoch",num_train_epochs= 10
                                  ,do_train=True,do_predict=True,logging_dir="/content/Docs",
                                  metric_for_best_model="accuracy",load_best_model_at_end = True,save_strategy="epoch",
                                  logging_first_step=True)

In [ ]:
from transformers import EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(5)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=train_dataset,
    eval_dataset = val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,3.269100,1.882471,0.492512
2,1.918200,1.487695,0.585691


In [ ]:
trainer.save_model("/content/drive/MyDrive/aa_model")

In [ ]:
test_outputs = trainer.predict(test_dataset)
test_outputs[2]

{'test_loss': 2.5211408138275146,
 'test_accuracy': 0.610410094637224,
 'test_runtime': 20.2072,
 'test_samples_per_second': 31.375,
 'test_steps_per_second': 3.959}

In [ ]:
val_outputs = trainer.predict(val_dataset)
val_outputs[2]

NameError: ignored

In [ ]:
train_outputs = trainer.predict(train_dataset)
train_outputs[2]

{'test_loss': 0.024055374786257744,
 'test_accuracy': 0.9930023325558147,
 'test_runtime': 96.3026,
 'test_samples_per_second': 31.162,
 'test_steps_per_second': 3.904}